# metaL: гомоиконичный интерпретатор

https://ru.wikibooks.org/wiki/metaL

глава из сборника: [Реализация языков программирования](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F)

## Фрейм

Язык `metaL` построен на базе **фреймов** Марвина Мински [minsky], которые были расширены возможностью хранить упорядоченный набор вложенных элементов -- необходимое свойство для представления программ на любых языках программирования в виде *атрибутных грамматик* и *AST-деревьев*. Внутренняя модель и семантика языка наиболее близки языку [FORTH](https://ru.wikibooks.org/wiki/FORTH) (Форт), из которого убраны все низкоуровневые элементы (прямая адресация памяти, машинные числа на стеке), и наоборот добавлены объекты как базовый элемент языка. Объектизация Форта позволяет реализовать виртуальную машину `metaL` на любом современном императивном языке программирования напрямую, не вводя лишние слои такие как эмуляция образа памяти, стек целых чисел и байт-код.

In [2]:
class Frame:

    def __init__(self,V):

        # тэг типа/класса (необходим для лексера -- библиотека PLY)
        self.type = self.__class__.__name__.lower()

        # имя фрейма или скалярное значение (число, строка,..)
        self.val  = V

        # слоты = атрибуты = ассоциативный массив = словарь (Форт)
        self.slot = {}

        # универсальный упорядоченный контейнер = вектор = стек (Форт)
        self.nest = []

* `.type` поле требуется библиотекой-генератором парсеров PLY для всех объектов, которые она может рассматривать как токены при лексическом анализе
* `.val` используется как имя фрейма, или для хранения атомарных данных типа чисел и строк
* `.slot` атрибуты фрейма, являются символическими ссылками на другие фреймы, и одновременно формируют ребра направленного (гипер)графа.
* `.nest` хранит упорядоченный набор фреймов, и представляет одновременно стек, вектор переменной длины, и очередь

### Гомоиконичные языки программирования

> **Гомоиконичность** (гомоиконность, англ. homoiconicity, homoiconic)\
свойство некоторых языков программирования, в которых *представление
программ является одновременно структурами данных* определенных в типах самого
языка, *доступных для просмотра и модификации*. Говоря иначе,
гомоиконичность -- это когда исходный *код программы* пишется
*как базовая структура данных*, и язык программирования знает, как
получить к ней доступ (в том числе в рантайме).

В гомоиконичном языке *метапрограммирование это основная методика*
разработки ПО, использующаяся в том числе и *для расширения языка* до
возможностей, нужных конкретному программисту.

В качестве первого примера всегда приводится язык Лисп, который был создан для
обработки данных, представленных в форме вложенных списков.
Лисп-программы тоже записываются, хранятся и выполняются в виде списков; в
результате получается, что программа во время работы может получить доступ к
своему собственному исходному коду, а также автоматически изменять себя «на
лету». Гомоиконичные языки, как правило, включают полную поддержку
*синтаксических макросов*, позволяющие программисту определять новые
синтаксические структуры, и выражать *преобразования программ* в компактной
форме.

### Текстовый дамп

Чтобы работать с фреймовыми гиперграфами, пользователю необходимо средство их просмотра, здесь мы определим несколько методов для вывода текстового дампа в виде дерева, а в следующем разделе -- графическое представление (которое тоже использует подобную рекурсию для своей работы).

In [7]:
class Frame(Frame): # хинт для Jupyter Notebook, расширение существующего класса
    
    ## текстовый дамп гиперграфа
    
    # преобразование вызывается при использовании print/str
    def __repr__(self):
        return self.dump()
    
    # полный дамп в виде дерева
    def dump(self,depth=0,prefix=''):
        
        # заголовок поддерева: отбивка табами на глубину рекурсии и <T:V> заголовок
        tree = self._pad(depth) + self.head(prefix)
        
        # останов бесконечной рекурсии при наличии циклов в графе
        if not depth: Frame._dump = [] # корень рекурсии -- нулевая глубина
        if self in Frame._dump: return tree + ' _/'
        else: Frame._dump.append(self)
        
        # slot{} рекурсивный обход подграфов по слотам (атрибутам)
        for i in self.slot:
            tree += self.slot[i].dump(depth+1,'%s = '%i)
            
        # nest[] рекурсивный обход подграфов по упорядоченному контейнеру
        idx = 0
        for j in self.nest:
            tree += j.dump(depth+1,'%i: '%idx) ; idx +=1
            
        # вызврат дампа текущего поддерева
        return tree
            
    # минимальный дамп: только <T:V> заголовок
    

In [6]:
print(Frame('Hello World'))

AttributeError: 'Frame' object has no attribute '_pad'

## Ссылки

[minsky]\
Marvin [Minsky A Framework for Representing Knowledge](https://courses.media.mit.edu/2004spring/mas966/Minsky%201974%20Framework%20for%20knowledge.pdf) 1974\
Марвин Минский [Фреймы для представления знаний](https://royallib.com/book/minskiy_marvin/freymi_dlya_predstavleniya_znaniy.html) М.: Мир, 1979

[lutz]\
Mark Lutz **Programming Python, 4th Edition** O'Reilly 2019\
Марк Лутц **Изучаем Python. Том 1** М.: Вильямс, 2019 [ozon](https://www.ozon.ru/context/detail/id/156082566/)